## Problem Statement
Find the minimum of the parabola $F(x) = 10 * x^2$ using Genetic Algorithm.
We can encode an individual in the population as a vector, `(x)` of size `(1,)`. \
We can also choose some artifical bounds for `(x)` \
So, $(x) = (0)$ would be the ideal individual since we know that the ideal solution is $\begin{pmatrix} x \\ F(x)\end{pmatrix} = \begin{pmatrix} 0 \\ 0\end{pmatrix}$

In [2]:
import numpy as np
def generate_population(param_size, pop_size, low_bound, high_bound, discrete = False):
    """
    Generates a random matrix of size (pop_size, param_size) with values in the given bounds. 
    Integer values are used if discrete = True
    """
    population = np.random.uniform(low= low_bound, high = high_bound, size=(pop_size, param_size))
    if discrete:
        population = np.round(population)
    return population

In [3]:
PARAM_SIZE = 1
POP_SIZE = 10
LOW_BOUND = -10
HIGH_BOUND = 10
population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
print(population.shape)
print(np.mean(population))

(10, 1)
0.7829076946945415


In [4]:
def calculate_fitness(total_pop):
    """ Calculates fitness using the given function f(x) = 10 *x^2
    Returns a (POP_SIZE, ) numpy array
    """       
    fitness = 10 * np.square(total_pop)
    return fitness


In [5]:
fitness = calculate_fitness(population)
print(fitness.shape)
print(fitness.T)

(10, 1)
[[5.52346566e+02 7.35797789e+02 4.36264724e-01 3.42470057e+01
  4.33153725e+01 5.07967389e+02 3.94553930e+02 1.39793393e+02
  5.28340255e+02 1.12590067e+00]]


In [6]:
def select_determinstic(t_pop, t_fitness, n_mating, n_params):
    """ Given current population t_pop, select n_mating number of parents using determinstic selection scheme
    based on t_fitness (the fitness of the population) 

    Returns parents and their fitness as a tuple (parent, fitness_of_parents)
    """    
    parents = np.empty((n_mating, n_params))
    
    best_fitness = np.argsort(t_fitness.flatten())[0:n_mating]
    parents = t_pop[best_fitness]
    return (parents, calculate_fitness(parents))

In [7]:
N_MATING = 4
parents, par_fitness = select_determinstic(population, fitness, N_MATING, PARAM_SIZE)
print(parents.shape)
print(parents)
print(par_fitness)

(4, 1)
[[-0.20886951]
 [ 0.33554443]
 [-1.85059465]
 [-2.08123455]]
[[ 0.43626472]
 [ 1.12590067]
 [34.24700566]
 [43.31537251]]


In [8]:
def crossover(t_parents, n_offspring, idx_crossover):
    """ Given a set of parents, combine them and return offspring vectors
    
    Returns the offsprings and their fitness
    """
    
    # Create an emppty vector
    offspring = np.empty((n_offspring, t_parents.shape[1]))

    # Fill in crossover details
    for i in range(n_offspring):
        parent1 = t_parents[i]
        parent2 = t_parents[i+1]
        offspring[i] = np.copy(parent2)
        offspring[i,idx_crossover:] = parent1[idx_crossover:]
        
    return (offspring, calculate_fitness(offspring))
def mutation(t_offspring, mutation_rate):
    """ Given a set of offsprings, introduce mutation in them
   
    Returns the mutated offsprings and their fitness
    """
    # Fill in details
    mutated_offspring = np.copy(t_offspring)
    
    random_values = np.random.uniform(low=-0.5, high=0.5, size=t_offspring.shape)
    coin_toss = np.random.uniform(low=0, high=1, size=t_offspring.shape)
    ind = np.nonzero(coin_toss > mutation_rate)
    print(ind)
    print(random_values.shape)
    mutated_offspring[ind] += random_values[ind]
        
    return (mutated_offspring, calculate_fitness(mutated_offspring))

In [9]:
N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
IDX_CROSSOVER = 0 # Fill in at which index you want crossover
PM = 0.5 # Mutation rate parameter

offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
offspring, offspring_fitness = mutation(offspring, 0.5)
print(offspring)
print(offspring_fitness)

(array([2]), array([0]))
(3, 1)
[[-0.20886951]
 [ 0.33554443]
 [-1.79554979]]
[[ 0.43626472]
 [ 1.12590067]
 [32.23999046]]


In [10]:
def environmental_selection(curr_population, offspring):
    """ Calculate total population (after constraint checking) fitness,
    rank accoridingly and select only the top POP_SIZE individuals to
    pass on to the next generation
    """ 

    # Fill in details
    t_total_pop = np.vstack((curr_population, offspring))
    new_pop, new_pop_fitness = select_determinstic(t_total_pop, calculate_fitness(t_total_pop), curr_population.shape[0], curr_population.shape[1])
    return new_pop

In [11]:
new_pop = environmental_selection(population, offspring)
print(new_pop)

[[-0.20886951]
 [-0.20886951]
 [ 0.33554443]
 [ 0.33554443]
 [-1.79554979]
 [-1.85059465]
 [-2.08123455]
 [ 3.73889546]
 [ 6.2813528 ]
 [-7.1271831 ]]


In [12]:
best_outputs = []
num_generations = 1000
curr_population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
overall_max_fitness = 99999

# Run many iterations
# You should also have another convergence check
for generation in range(num_generations):
    print("Generation : ", generation)

    # Measuring the fitness of each chromosome in the population.
    fitness = calculate_fitness(curr_population)
    print('Fitness')
    print(fitness.shape)
    print(fitness.T)

    min_fitness = np.min(fitness)

    # The best result in the current iteration.
    print("Best result in current iteration {0} compared to overall {1}".format(min_fitness, min(min_fitness, overall_max_fitness)))
    best_outputs.append(min_fitness)
    
    # Selecting the best parents in the population for mating.
    N_MATING = 6
    parents, par_fitness = select_determinstic(curr_population, fitness, N_MATING, PARAM_SIZE)
    print("Parents")
    print(parents.shape)
    print(parents)
    print(par_fitness)
#     parents, _ = select_stochastic(curr_population, fitness)
    
    # print("Parents")
    # print(parents)
    N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
    IDX_CROSSOVER = 0 # Fill in at which index you want crossover
    PM = 0.5 # Mutation rate parameter

    offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
    offspring, offspring_fitness = mutation(offspring, PM)
    print("Mutation")
    print(offspring)
    print(offspring_fitness)

    # print("Mutation")
    # print(offspring_mutation)

    # Environmental selection
    curr_population = environmental_selection(curr_population, offspring)
              
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = calculate_fitness(curr_population)
print(fitness)
# Then return the index of that solution corresponding to the best fitness.
max_idx = np.argmin(fitness)

print("Best solution : ", curr_population[max_idx, :])
print("Best solution fitness : ", fitness[max_idx])

Generation :  0
Fitness
(10, 1)
[[5.70702577e+02 1.57822476e+02 7.10514433e+02 1.72302379e+02
  6.23326355e+02 4.19631701e+01 8.09450862e+02 2.61288189e+02
  3.58820510e+02 7.92673853e-04]]
Best result in current iteration 0.0007926738527694955 compared to overall 0.0007926738527694955
Parents
(6, 1)
[[-0.00890322]
 [-2.0484914 ]
 [ 3.97268771]
 [-4.15093217]
 [ 5.11163564]
 [ 5.99016285]]
[[7.92673853e-04]
 [4.19631701e+01]
 [1.57822476e+02]
 [1.72302379e+02]
 [2.61288189e+02]
 [3.58820510e+02]]
(array([0, 2, 3, 4]), array([0, 0, 0, 0]))
(5, 1)
Mutation
[[-0.32294161]
 [-2.0484914 ]
 [ 4.0605412 ]
 [-3.93450127]
 [ 5.32752707]]
[[  1.04291284]
 [ 41.96317007]
 [164.87994816]
 [154.80300275]
 [283.82544691]]
Generation :  1
Fitness
(10, 1)
[[7.92673853e-04 1.04291284e+00 4.19631701e+01 4.19631701e+01
  1.54803003e+02 1.57822476e+02 1.64879948e+02 1.72302379e+02
  2.61288189e+02 2.83825447e+02]]
Best result in current iteration 0.0007926738527694955 compared to overall 0.000792673852769

# Results
After running the above code, we get:
```
Best solution :  [0.00021285]
Best solution fitness :  [4.53041994e-07]
```
Which is quite close to the correct solution of $x=0$ to minimize the function $F(x) = 10 * x^2$